In [1]:
import semantic_kernel as sk
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
print(os.getenv("OPENAI_API_BASE"))


https://alkopenai2.openai.azure.com/


In [2]:
# for this example I want a valid logger
import logging

# Create a logger
logger = logging.getLogger("SK")

# Set the logging level to INFO
logger.setLevel(logging.DEBUG)

# Create a console handler
console_handler = logging.StreamHandler()

# Set the level of the console handler to INFO
console_handler.setLevel(logging.DEBUG)

# Create a formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Set the formatter for the console handler
console_handler.setFormatter(formatter)

# Add the console handler to the logger
logger.addHandler(console_handler)


In [3]:
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextCompletion,
)

kernel = sk.Kernel(log=logger)
kernel.add_chat_service(
    "chat_completion",
    AzureChatCompletion(
        "GPT42",
        os.getenv("OPENAI_API_BASE"),
        os.getenv("OPENAI_API_KEY")
    ),
)

In [4]:
# Now we need to import the plugin
from plugins.AudioVideoPlugin.AudioVideo import AudioVideo

# Now you can import the plugin importing skill directly from the function you declared
# in the plugin directory. The import_skill does not need the path, it only need an
# instance of the skill and the name of the skill
extractaudio_plugin = kernel.import_skill(AudioVideo(), skill_name="AudioVideoPlugin")

plugins_directory = "./plugins"

# Import the OrchestratorPlugin from the plugins directory.
publishing_plugin = kernel.import_semantic_skill_from_directory(
    plugins_directory, "PublishingPlugin"
)


2023-12-03 08:51:19,831 - SK - DEBUG - Importing skill AudioVideoPlugin
2023-12-03 08:51:19,832 - SK - DEBUG - Methods imported: 2
2023-12-03 08:51:19,833 - SK - DEBUG - Extracting blocks from template: I will give you a transcript of a video. The transcript contains phrases prefixed by the timestamp where the phrase starts. I want you to identify between three and ten main sections of the video. For each section you will create a brief title prefixed with the start timestamp of the section, obtained by the first phrase of the section. 

EXAMPLE ANSWER
00:00 - Title 1
00:33 - Title 2
01:23 - Title 3

[DATA]
{{$input}}


In [5]:
from pprint import pprint
# want to print all the keys of extractaudio_plugin that is a dictionary

pprint (extractaudio_plugin["ExtractAudio"])
pprint (extractaudio_plugin["TranscriptTimeline"])
pprint (publishing_plugin["VideoTimelineCreator"])

In [6]:
# you can verify if cuda is available.
import torch
torch.cuda.is_available()

True

In [7]:
# Now we can use the skill
# Run the Sqrt function with the context.
result = await kernel.run_async(
    extractaudio_plugin["ExtractAudio"],
    extractaudio_plugin["TranscriptTimeline"],
    publishing_plugin["VideoTimelineCreator"],
    input_str="C:\\temp\\ssh.mp4",
)


Extracting auio file from video C:\temp\ssh.mp4
Extracting transcript from audio file C:\temp\ssh.wav
Using device: cuda:0 to run whisper


2023-12-03 08:52:12,926 - SK - DEBUG - Rendering string template: I will give you a transcript of a video. The transcript contains phrases prefixed by the timestamp where the phrase starts. I want you to identify between three and ten main sections of the video. For each section you will create a brief title prefixed with the start timestamp of the section, obtained by the first phrase of the section. 

EXAMPLE ANSWER
00:00 - Title 1
00:33 - Title 2
01:23 - Title 3

[DATA]
{{$input}}
2023-12-03 08:52:12,927 - SK - DEBUG - Extracting blocks from template: I will give you a transcript of a video. The transcript contains phrases prefixed by the timestamp where the phrase starts. I want you to identify between three and ten main sections of the video. For each section you will create a brief title prefixed with the start timestamp of the section, obtained by the first phrase of the section. 

EXAMPLE ANSWER
00:00 - Title 1
00:33 - Title 2
01:23 - Title 3

[DATA]
{{$input}}
2023-12-03 08:52

extracted 47 audio segments


In [8]:

print(result)

00:00 - Introduction to Yubikey and SSH Key Issue on Windows
00:39 - Explanation of SSH Version Problem on Windows
01:15 - Questioning the SSH Update Process on Windows
01:35 - Understanding the SSH Version Discrepancy on GitHub
01:57 - Explanation of Microsoft's Change in SSH Support
02:18 - Solution to Update SSH Version on Windows
03:33 - Final Reminder and Conclusion of the Video
